In [1]:
%matplotlib widget

import sys
sys.path.append("./../scripts")
import PandemiXFunctions as pf
import PandemiXModelFunctions as pmf


import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }

In [2]:
import importlib 
# importlib.reload(PandemiXModelFunctions)
importlib.reload(pmf)

<module 'PandemiXModelFunctions' from 'c:\\Users\\rakrpe\\OneDrive - Roskilde Universitet\\Documents\\PandemiX\\GithubRepos\\PandemiX\\modelling\\PandemiXModelFunctions.py'>

In [3]:
# Setup model

# Parameters
gamma_I = 1/7
gamma_Y = gamma_I 
beta_I = 1.5*gamma_I 
beta_Y = 3*gamma_Y
allPars = [beta_I,gamma_I,beta_Y,gamma_Y]

# Initial conditions
I_0 = 0.01
Y_0 = 0
R_0 = 0
S_0 = 1 - R_0 - I_0 - Y_0
init = [S_0,I_0,Y_0]

tEnd = 500
curScheme = pmf.Scheme('SIYR',init,allPars,0,tEnd)

In [4]:
# Base simulation
curScheme.simulate(tRes=1000)

resBase = curScheme.result 
ts = resBase.t
Ss = resBase.y[0,:]
Is = resBase.y[1,:]
Ys = resBase.y[2,:]
Rs = 1 - Ss - Is - Ys

fig,ax1 = plt.subplots()
ax1.plot(ts,Ss)
ax1.plot(ts,Is)
ax1.plot(ts,Ys)
ax1.plot(ts,Rs)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Make a scheme with added Y post-epidemic
Sch1 = curScheme.copy()
testChange = pmf.Change(20,AddVariables = [-0.001,0,0.001])
Sch1.addChange(testChange)
Sch2 = curScheme.copy()
testChange = pmf.Change(160,AddVariables = [-0.001,0,0.001])
Sch2.addChange(testChange)



In [6]:
# Run simulation

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(ts,Ss,color='b')
ax1.plot(ts,Rs,color='g')
ax2.plot(ts,Is,color='xkcd:orange')
ax2.plot(ts,Ys,color='r')
ax2.plot(ts,Is+Ys,color='k')

Sch1.simulate()

resSch1 = Sch1.result 
ts1 = resSch1.t
Ss1 = resSch1.y[0,:]
Is1 = resSch1.y[1,:]
Ys1 = resSch1.y[2,:]
Rs1 = 1 - Ss1 - Is1 - Ys1
ax1.plot(ts1,Ss1,':',color='b')
ax1.plot(ts1,Rs1,':',color='g')
ax2.plot(ts1,Is1,':',color='xkcd:orange')
ax2.plot(ts1,Ys1,':',color='r')
ax2.plot(ts1,Is1+Ys1,':',color='k')


Sch2.simulate()

resSch2 = Sch2.result 
ts2 = resSch2.t
Ss2 = resSch2.y[0,:]
Is2 = resSch2.y[1,:]
Ys2 = resSch2.y[2,:]
Rs2 = 1 - Ss2 - Is2 - Ys2
ax1.plot(ts2,Ss2,'--',color='b')
ax1.plot(ts2,Rs2,'--',color='g')
ax2.plot(ts2,Is2,'--',color='xkcd:orange')
ax2.plot(ts2,Ys2,'--',color='r')
ax2.plot(ts,Is+Ys,color='k')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:


def getFinalDiffAndSumY(t):
    
    
    Sch1 = curScheme.copy()
    testChange = pmf.Change(t,AddVariables = [-0.001,0,0.001])
    Sch1.addChange(testChange)
    # Run simulation
    Sch1.simulate()

    resSch1 = Sch1.result 
    ts1 = resSch1.t
    Ss1 = resSch1.y[0,:]
    Is1 = resSch1.y[1,:]
    Ys1 = resSch1.y[2,:]
    Rs1 = 1 - Ss1 - Is1 - Ys1
        
    finalDiff = Rs1[-1] - Rs[-1]
    tRange = np.arange(0,ts[-1])
    curDailyYs = np.interp(tRange,ts1,Ys1)
    
    sumY = np.sum(curDailyYs)
    
    return finalDiff,sumY
    

# getFinalDiffAndSumY(10)
tToTest = np.arange(10,450,20)

fds = []
sumys = []
for curT in tToTest:
    this_fd,this_sy = getFinalDiffAndSumY(curT)
    
    fds.append(this_fd)
    sumys.append(this_sy)

fds = np.array(fds)
sumys = np.array(sumys)

In [8]:
fig,(ax1,ax2) = plt.subplots(2,1)
ax1.plot(tToTest,fds)
ax2.plot(tToTest,sumys)
ax1.set_ylabel('Øgning i andel immune')
ax2.set_ylabel('Sum af superior variant')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Sum af superior variant')

In [9]:

# print(np.sum(Ys1))
# print(np.sum(curDaily))

# plt.figure()
# plt.plot(tRange,curDaily,'.-')
# plt.plot(ts1,Ys1,'.-')

# Changing restrictions
Different scenarios in which a superior variant is introduced at some point in an epidemic, and R_0 for both variants are changed (e.g. due to changing restrictions)


In [10]:
# Setup base model

# Parameters
gamma_I = 1/7
gamma_Y = gamma_I 
beta_I = 1.25*gamma_I 
beta_Y = 1.5*gamma_Y
allPars = [beta_I,gamma_I,beta_Y,gamma_Y]

# Initial conditions
I_0 = 0.001
Y_0 = 0
R_0 = 0
S_0 = 1 - R_0 - I_0 - Y_0
init = [S_0,I_0,Y_0]

tEnd = 500
curScheme = pmf.Scheme('SIYR',init,allPars,0,tEnd,[])

betaMultiplier = 1.25
RestrictionChange = pmf.Change(200,MultiplyParameters=[betaMultiplier,1,betaMultiplier,1])
curScheme.addChange(RestrictionChange)

In [11]:
schAdd1 = curScheme.copy()
schAdd2 = curScheme.copy()

newInf = 0.001
Change_Add1 = pmf.Change(150,AddVariables=[-newInf,0,newInf])
Change_Add2 = pmf.Change(220,AddVariables=[-newInf,0,newInf])

schAdd1.addChange(Change_Add1)
schAdd2.addChange(Change_Add2)

schAdd1.sortChanges()
# schAdd1.sortChanges()

In [12]:
schAdd1.Changes

In [13]:
# Run simulations
curScheme.simulate(tRes=1000)
schAdd1.simulate(tRes=1000)
schAdd2.simulate(tRes=1000)

resBase = curScheme.result 
ts = resBase.t
Ss = resBase.y[0,:]
Is = resBase.y[1,:]
Ys = resBase.y[2,:]
Rs = 1 - Ss - Is - Ys

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(ts,Ss,color='b')
ax1.plot(ts,Rs,color='g')
ax2.plot(ts,Is,color='xkcd:orange')
ax2.plot(ts,Ys,color='r')

ts1 = schAdd1.result.t
Ss1 = schAdd1.result.y[0,:]
Is1 = schAdd1.result.y[1,:]
Ys1 = schAdd1.result.y[2,:]
Rs1 = 1 - Ss1 - Is1 - Ys1
ax1.plot(ts1,Ss1,':',color='b')
ax1.plot(ts1,Rs1,':',color='g')
ax2.plot(ts1,Is1,':',color='xkcd:orange')
ax2.plot(ts1,Ys1,':',color='r')

ts2 = schAdd2.result.t
Ss2 = schAdd2.result.y[0,:]
Is2 = schAdd2.result.y[1,:]
Ys2 = schAdd2.result.y[2,:]
Rs2 = 1 - Ss2 - Is2 - Ys2
ax1.plot(ts2,Ss2,'--',color='b')
ax1.plot(ts2,Rs2,'--',color='g')
ax2.plot(ts2,Is2,'--',color='xkcd:orange')
ax2.plot(ts2,Ys2,'--',color='r')


fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Subsequent epidemics, with different timing of loosening restrictions

In [14]:
# Setup base model

# Parameters
gamma_I = 1/7
gamma_Y = gamma_I 
beta_I = 1.25*gamma_I 
beta_Y = 1.5*gamma_Y
allPars = [beta_I,gamma_I,beta_Y,gamma_Y]

# Initial conditions
I_0 = 0.001
Y_0 = 0
R_0 = 0
S_0 = 1 - R_0 - I_0 - Y_0
init = [S_0,I_0,Y_0]

tEnd = 500
curScheme = pmf.Scheme('SIYR',init,allPars,0,tEnd,[])

newInf = I_0
Change_Add = pmf.Change(250,AddVariables=[-newInf,0,newInf])

curScheme.addChange(Change_Add)

In [15]:
schMult1 = curScheme.copy()
schMult2 = curScheme.copy()

betaMultiplier = 2
Change_Mult1 = pmf.Change(200,MultiplyParameters=[betaMultiplier,1,betaMultiplier,1])
Change_Mult2 = pmf.Change(300,MultiplyParameters=[betaMultiplier,1,betaMultiplier,1])

schMult1.addChange(Change_Mult1)
schMult2.addChange(Change_Mult2)

schMult1.sortChanges()
# schMult1.sortChanges()

In [16]:
# Run simulations
curScheme.simulate(tRes=1000)
schMult1.simulate(tRes=1000)
schMult2.simulate(tRes=1000)

resBase = curScheme.result 
ts = resBase.t
Ss = resBase.y[0,:]
Is = resBase.y[1,:]
Ys = resBase.y[2,:]
Rs = 1 - Ss - Is - Ys

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(ts,Ss,color='b')
ax1.plot(ts,Rs,color='g')
ax2.plot(ts,Is,color='xkcd:orange')
ax2.plot(ts,Ys,color='r')


ts1 = schMult1.result.t
Ss1 = schMult1.result.y[0,:]
Is1 = schMult1.result.y[1,:]
Ys1 = schMult1.result.y[2,:]
Rs1 = 1 - Ss1 - Is1 - Ys1
ax1.plot(ts1,Ss1,':',color='b')
ax1.plot(ts1,Rs1,':',color='g')
ax2.plot(ts1,Is1,':',color='xkcd:orange')
ax2.plot(ts1,Ys1,':',color='r')

ts2 = schMult2.result.t
Ss2 = schMult2.result.y[0,:]
Is2 = schMult2.result.y[1,:]
Ys2 = schMult2.result.y[2,:]
Rs2 = 1 - Ss2 - Is2 - Ys2
ax1.plot(ts2,Ss2,'--',color='b')
ax1.plot(ts2,Rs2,'--',color='g')
ax2.plot(ts2,Is2,'--',color='xkcd:orange')
ax2.plot(ts2,Ys2,'--',color='r')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


$$\dot{H} = r_{chr} \cdot I - u \cdot H(t)$$
or 
$$\dot{H} + u \cdot H(t) = r_{chr} \cdot I $$

Has solution:

$\mu(t) = e^{u t}$

So:
$e^{ut}H(t) + k = \frac{1}{u} e^{ut} r_{chr} I$

Or, in other words:
$$ H(t) = \frac{r_{chr} I}{u} + k e^{-ut}$$

In [17]:
# Calculate the corresponding hospitalizations

CHR1 = 0.1
CHR2 = 0.2
outH = 0.01
k = 0

testT = ts
testI = Is

testH = (CHR1*testI/outH) + k * np.exp(-outH*testT)


HI_1 = (CHR1*Is1/outH) + k * np.exp(-outH*ts1)
HY_1 = (CHR2*Ys1/outH) + k * np.exp(-outH*ts1)
HI_2 = (CHR1*Is2/outH) + k * np.exp(-outH*ts2)
HY_2 = (CHR2*Ys2/outH) + k * np.exp(-outH*ts2)

plt.figure()
plt.plot(testT,testH,'k')
plt.plot(ts1,HI_1+HY_1,'b:')
plt.plot(ts2,HI_2+HY_2,'m--')
# plt.plot(ts1,HY_1,':')
 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


$$\dot{H} = r_{chr} \cdot I(t) - u \cdot H(t)$$
or 
$$\dot{H} - u \cdot H(t) = r_{chr} \cdot I(t) $$

Has solution:

$\mu(t) = e^{-u t}$

So:
$e^{-ut}H(t) + k = \int e^{-ut} r_{chr} I(t) dt $

Or, in other words:
$$ H(t) = e^{ut} \left( r_{chr} \int e^{-ut} I(t) dt +k \right)